In [1]:
import sys,os
import uproot4 as uproot
import awkward1 as ak
import json
import numpy as np
import matplotlib.pyplot as plt
from fcc_python_tools.locations import loc
from fcc_python_tools import kinematics
from particle import literals as lp
from fcc_python_tools import plotting

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Roman']})
rc('text', usetex=True)

file = uproot.open(f"{loc.DATA}/FCCDelphesOutput.root")
tree = file['events']

Access the generated truth-level particles from `genParticles`:

In [3]:
g_container = "genParticles.core"
g = tree.arrays(filter_name=f"{g_container}*",how="zip")

Access the reco hadrons from `efcharged`:

In [15]:
h_container = "efcharged.core"
h = tree.arrays(filter_name=f"{h_container}*",how="zip")

<Array [{'efcharged.core': [, ... ] type='10000 * {"efcharged.core": var * {"pdg...'>

Access the particle to MC particle info for the charged hadrons:

In [25]:
h_toMC_container = "efchargedToMC\#1"
h_toMC = tree.arrays(filter_name=f"{h_toMC_container}*",how="zip")